# Toxic Comment Prediction 

In [ ]:
!pip install ktrain

In [3]:
! git clone https://github.com/laxmimerit/Toxic-Comment.git

Cloning into 'Toxic-Comment'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [4]:
import pandas as pd
import ktrain
from ktrain import text

In [9]:
PATH = "/content/Toxic-Comment/train.csv"

train, val , preproc = text.texts_from_csv(PATH,'comment_text' ,label_columns=['toxic','severe_toxic',
                                                        'obscene', 'threat','insult','identity_hate'],
                    ngram_range =1 , max_features = 50000, maxlen =150 )

detected encoding: utf-8 (if wrong, set manually)
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
98018       0             0        0       0       0              0
34913       0             0        0       0       0              0
68030       0             0        0       0       0              0
109645      0             0        0       0       0              0
63186       0             0        0       0       0              0
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
8518        0             0        0       0       0              0
115183      0             0        0       0       0              0
42515       1             0        0       0       0              0
108424      0             0        0       0       0              0
15122       0             0        0       0       0  

In [6]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [10]:
model = text.text_classifier('fasttext',train,preproc)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
done.


In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 64)           3200000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 64)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [11]:
learner = ktrain.get_learner(model, train, val)

In [12]:
%time learner.autofit(0.001,2)



begin training using triangular learning rate policy with max lr of 0.001...
Epoch 1/2
4488/4488 [==============================] - 155s 34ms/step - loss: 0.3063 - accuracy: 0.5041 - val_loss: 0.0550 - val_accuracy: 0.9945
Epoch 2/2
4488/4488 [==============================] - 145s 32ms/step - loss: 0.0655 - accuracy: 0.9879 - val_loss: 0.0521 - val_accuracy: 0.9945
CPU times: user 7min 45s, sys: 18.6 s, total: 8min 3s
Wall time: 5min


In [13]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [14]:
predictor.predict(['Yor are amazing'])

[[('toxic', 0.065121606),
  ('severe_toxic', 1.7573222e-05),
  ('obscene', 0.005370948),
  ('threat', 0.00022455186),
  ('insult', 0.00975184),
  ('identity_hate', 0.00083664455)]]

In [15]:
predictor.predict(['I hate you , you are so crazy'])

[[('toxic', 0.94043905),
  ('severe_toxic', 0.07688414),
  ('obscene', 0.58740044),
  ('threat', 0.048209522),
  ('insult', 0.5720059),
  ('identity_hate', 0.1387509)]]

In [16]:
predictor.save('/content/fasttext')

In [17]:
predictor = ktrain.load_predictor('fasttext')

def get_prediction(x):
	pred = predictor.predict([x])
	return pred[0]

In [18]:
data = 'You are so stopid'

In [19]:
pred = get_prediction(data)

In [20]:
pred

[('toxic', 0.39305624),
 ('severe_toxic', 0.0012070052),
 ('obscene', 0.047301333),
 ('threat', 0.004651728),
 ('insult', 0.09258503),
 ('identity_hate', 0.009582117)]

In [21]:
import json

In [22]:
d = dict(pred)
for key in d:
    d[key] = str(d[key])

In [23]:
d

{'identity_hate': '0.009582117',
 'insult': '0.09258503',
 'obscene': '0.047301333',
 'severe_toxic': '0.0012070052',
 'threat': '0.004651728',
 'toxic': '0.39305624'}

In [24]:
json.dumps(d)

'{"toxic": "0.39305624", "severe_toxic": "0.0012070052", "obscene": "0.047301333", "threat": "0.004651728", "insult": "0.09258503", "identity_hate": "0.009582117"}'